In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/検索技術_最終課題/datas&models

Mounted at /content/drive
/content/drive/MyDrive/検索技術_最終課題/datas&models


In [2]:
pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.1 MB/s 


In [4]:
from janome.tokenizer import Tokenizer
import zipfile
import os.path, urllib.request as req
import glob
import os
import tensorflow_datasets as tfds
from gensim.models import word2vec
import re

In [5]:
# データセットの取得
ds = tfds.load('wiki40b/ja', split='test')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/20 [00:00<?, ? file/s]


Dataset wiki40b downloaded and prepared to /root/tensorflow_datasets/wiki40b/ja/1.3.0. Subsequent calls will reuse this data.


In [6]:
def create_text(tf_data):
    start_paragraph = False
    for text in wiki['text'].decode().split('\n'):
      if start_paragraph:
        text = text.replace('_NEWLINE_', '\n') # _NEWLINE_は削除
        start_paragraph = False
      if text == '_START_PARAGRAPH_': # _START_PARAGRAPH_のみ取得
        start_paragraph = True
    return text

In [7]:
def mp_analysis(text, wakati_file):  # 形態素解析
  t = Tokenizer()
  results = []
  
  # テキストを一行ずつ処理する
  lines = text.split("\r\n")
  for line in lines:
    s = line
    s = re.sub(r'（.+?）', '', s) # （ ）を削除
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'［＃.+?］', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする)
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本系を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    
  # 書き込み先テキストを開く
  with open(wakati_file, 'a', encoding='utf-8') as fp:
    fp.write("\n".join(results))

In [8]:
# 書き込み先テキストの初期化
wakati_file = open('wiki.wakati','w')
wakati_file.truncate(0)

# テキストファイルの読み込み
for wiki in list(ds.as_numpy_iterator())[0:1000]:
  text = create_text(wiki)
  mp_analysis(text, 'wiki.wakati')
    
# Word2Vecでモデルを作成
wiki_data = word2vec.LineSentence('wiki.wakati')
wiki_model = word2vec.Word2Vec(wiki_data, size=200, window=10, hs=1, min_count=2, sg=1)
wiki_model.save('wiki.model')
print('ok')

ok


In [9]:
# ZIPファイルをダウンロード
local = ["799_ruby_6024.zip", "53859_ruby_47309.zip", "50670_ruby_37831.zip", "43606_ruby_17004.zip", "47061_ruby_28378.zip", "49922_ruby_48217.zip", "49209_ruby_32175.zip", "45245_ruby_21882.zip", "54905_ruby_49755.zip", "3325_ruby_6457.zip"]

# 書き込み先テキストの初期化
wakati_file = open('book.wakati','w')
wakati_file.truncate(0)

0

In [10]:
for zp in local:
  # ZIPファイル内のテキストファイルを読む
  zf = zipfile.ZipFile(zp, 'r') # zipファイルを読む
  infos = zf.namelist()
  file_name = []

  for extract in infos:
    if '.txt' in extract:
      info = extract
      info.replace('[','')
      info.replace(']','')
      file_name.append(info)

  for data in file_name:
    fp= zf.open(data, 'r') # アーカイブ内のテキストを読む
    bindata = fp.read()
    book_text = bindata.decode('shift_jis') # テキストがShift_JISなのでデコード
    
    # テキストファイルの読み込み
    mp_analysis(book_text, 'book.wakati')
  
# Word2Vecでモデルを作成
book_data = word2vec.LineSentence('book.wakati')
book_model = word2vec.Word2Vec(book_data, size=200, window=10, hs=1, min_count=2, sg=1)
book_model.save('book.model')
print('ok')


ok


In [11]:
wiki_model = word2vec.Word2Vec.load('wiki.model')
book_model = word2vec.Word2Vec.load('book.model')
search_word = ['国', '人', '夢', '時']

print('wiki_model:')

for word in search_word:
    wiki_words = wiki_model.most_similar(positive=[word])
    n = [w[0] for w in wiki_words]
    print(word,"=", ",".join(n))

print('book_model')
for word in search_word:
    book_words = book_model.most_similar(positive=[word])
    n = [w[0] for w in book_words]
    print(word,"=", ",".join(n))

wiki_model:
国 = 交渉,軍勢,共和,後藤,塞ぐ,征服,従軍,全会,陸奥,マロリー
人 = セルビア,クロアチア,日系,レスラー,アフリカ,子ども,ペルシア,授かる,突破,ディストモ
夢 = リンク,嵐,ペーパー,すく,花街,サスペンス,1900,根本,クリップ,本名
時 = 小学生,年生,干潮,組み立てる,囲碁,遅れる,マイラーズカップ,鎧,83,。5
book_model
国 = 人民,日本,全国,棄てる,諸国,アジヤ,食客,恥辱,結ぶ,気力
人 = 活発,道理,君子,発する,冊,みな,界,厭う,心得,聞見
夢 = 深い,立派,吹き出る,光景,春,呑気,刹那,畳,優しい,注ぐ
時 = 縛る,思案,並ぶ,立膝,尽きる,頂,違える,女房,餅屋,夫


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
